In [1]:
%load_ext autoreload 
%autoreload 2

In [27]:

import torch
import wandb
from rnn_reserving.data_import import read_and_process_data
from rnn_reserving.config import TrainingConfig, ModelConfig
from rnn_reserving.dataloader import make_loaders

from rnn_reserving.models import GRUInsurance

from rnn_reserving.trainer import Trainer

In [10]:

config = TrainingConfig(
    batch_size=32,
    shuffle_train=True,
    early_stopping_patience=10,
)

model_config = ModelConfig()
target_cols = ['paid_loss_ratio']

feature_cols = []

In [11]:
print(config)

TrainingConfig(data_path='./data/raw/ppauto_pos.csv', log_file='./logs/training.log', seed=42, device='cpu', model_name='gru_model', experiment_name='rnn_reserving_experiment', run_name='run_1', feature_cols=[], target_cols=['paid_loss_ratio'], epochs=100, batch_size=32, learning_rate=0.001, weight_decay=1e-05, shuffle_train=True, grad_clip=1.0, use_mixed_precision=True, scheduler_type='cosine', scheduler_patience=10, scheduler_factor=0.5, early_stopping_patience=10, early_stopping_min_delta=0.0001, checkpoint_dir='./checkpoints', save_best_only=True, save_frequency=5, log_interval=10, use_wandb=True, wandb_project='timeseries-forecasting', deterministic=False, num_workers=4)


In [5]:
train_loader, val_loader = make_loaders(config)

In [6]:
train_loader.batch_size

32

In [17]:
model = GRUInsurance(model_config)

In [21]:

def metrics_fn(output: torch.Tensor, target: torch.Tensor) -> Dict[str, float]:
    """Compute metrics given model output and target."""
    mse = nn.MSELoss()(output, target).item()
    mae = nn.L1Loss()(output, target).item()
    return {
        'mse': mse,
        'mae': mae
    }


In [24]:
criterion = torch.nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser, T_max=50)  # 50 epochs cycle



In [31]:
trainer = Trainer(
    model,
    train_loader,
    val_loader,
    criterion,
    optimiser,
    config,
    scheduler=scheduler,
    metrics_fn=metrics_fn,
)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\TobyCook\_netrc
wandb: Currently logged in as: toby-cook (toby-cook-toby-cook) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [32]:
trainer.train()

2025-11-11 21:37:56,455 - rnn_reserving.utils - INFO - Starting training on cpu
2025-11-11 21:37:56,455 - rnn_reserving.utils - INFO - Starting training on cpu
2025-11-11 21:37:56,455 - rnn_reserving.utils - INFO - Starting training on cpu
2025-11-11 21:37:56,464 - rnn_reserving.utils - INFO - Config: {'data_path': './data/raw/ppauto_pos.csv', 'log_file': './logs/training.log', 'seed': 42, 'device': 'cpu', 'model_name': 'gru_model', 'experiment_name': 'rnn_reserving_experiment', 'run_name': 'run_1', 'feature_cols': [], 'target_cols': ['paid_loss_ratio'], 'epochs': 100, 'batch_size': 32, 'learning_rate': 0.001, 'weight_decay': 1e-05, 'shuffle_train': True, 'grad_clip': 1.0, 'use_mixed_precision': True, 'scheduler_type': 'cosine', 'scheduler_patience': 10, 'scheduler_factor': 0.5, 'early_stopping_patience': 10, 'early_stopping_min_delta': 0.0001, 'checkpoint_dir': './checkpoints', 'save_best_only': True, 'save_frequency': 5, 'log_interval': 10, 'use_wandb': True, 'wandb_project': 'timese

ValueError: too many values to unpack (expected 2, got 3)